---
title: pyOpenSci Current Software Review Stats
subtitle: pyOpenSci Peer Review Summary Stats
author:
  - name: Leah Wasser
    affiliations: pyOpenSci
    orcid: 0000-0002-7859-8394
    email: leah@pyopensci.org
license:
  code: MIT
date: 2024/06/20
---


* https://github.com/ryantam626/jupyterlab_code_formatter

This is a workflow that colates all GitHub issues associated with our reviews. 

Questions i have

* How to add figure captions and alt text
* 

In [1]:
from datetime import datetime

import altair as alt
import pandas as pd

from pyosmeta import ProcessIssues
from pyosmeta.github_api import GitHubAPI

In [2]:
from functions import parse_single_issue

In [3]:
# Get all issues from GitHub software-submission repo, Return df with labels, title, date_opened and closed and total time open in days
github_api = GitHubAPI(
    org="pyopensci",
    repo="software-submission",
)

process_review = ProcessIssues(github_api)
issues = process_review.return_response()

all_issues = []
for issue in issues:
    all_issues.append(parse_single_issue(issue))

df = pd.DataFrame(all_issues)

# Remove issues that are unlabeled or say help wanted
valid_issues = df[
    ~(
        (df["labels"].apply(len) == 0)
        | df["labels"].apply(lambda x: "help wanted" in x or "Help Request" in x)
    )
]

# Total presubmissions - get the total number of pre-submission inquiries (all time)
total_presubmissions = valid_issues[
    valid_issues["labels"].apply(lambda x: "presubmission" in x)
]


In [4]:
# Get all currently open presubmissions 
open_presubmissions = total_presubmissions[total_presubmissions['date_closed'].isna()]
total_open = len(open_presubmissions)

## Currently open software presubmission inquiries

Can some of these be closed?  

* it could be useful to grab the most recent comments on each
* It would also be useful to grab the gh usernames of all people involved in the discussion and credit them. So for one i see astropy editors + alex being involved.

There are **{eval}`total_open` presubmission requests** currently open. 

In [5]:
cleaned_open_presubmissions = open_presubmissions.drop(columns=['date_closed'])
cleaned_open_presubmissions

,labels,header_text,date_opened,days_open
0,[presubmission],Solar Data Tools pre-submission inquiry,2024-06-28 23:12:55,-1.0
1,[presubmission],Presubmission Inquiry for MontePy,2024-06-17 18:07:13,11.0
13,[presubmission],Presubmission Inquiry for gentropy,2024-05-24 14:41:14,35.0
14,[presubmission],Presubmission Inquiry for GALAssify: A Python ...,2024-05-24 10:37:47,35.0
51,"[presubmission, ⌛ pending-maintainer-response]",WasteAndMaterialFootprint - presubmission enquiry,2023-12-30 17:23:18,181.0


In [6]:
total_submissions = valid_issues[
    valid_issues["labels"].apply(lambda x: "presubmission" not in x)
]

open_submissions = total_submissions[total_submissions['date_closed'].isna()]

## Current open softwrae review submissions 

Next we explore the currently open issues.

pyOpenSci currently has **{eval}`total_open`** total open submissions.

* x of these are in active review
* x of these are in pre-review
* x of these are being submitted to JOSS
* x of these have been approvved and are a part of our ecosystem 

In [11]:
final_open_submissions = open_submissions.drop(columns=['date_closed'])
final_open_submissions

,labels,header_text,date_opened,days_open
2,[0/seeking-editor],Great Tables submission,2024-06-14 19:55:59,13.0
3,"[0/pre-review-checks, New Submission!]",Stingray Submission,2024-06-14 12:59:47,14.0
9,[0/seeking-editor],Fluidimage submission,2024-05-30 12:53:48,29.0
22,"[3/reviewers-assigned, astropy]",astrodata,2024-05-13 23:48:03,45.0
23,[1/editor-assigned],QuadratiK Submission,2024-05-13 21:23:44,45.0
24,[0/seeking-editor],PyPartMC,2024-05-03 18:22:59,56.0
26,[0/pre-review-checks],Dolphot-LC Software Submission,2024-05-01 02:37:44,58.0
27,"[0/pre-review-checks, New Submission!, on-hold]",Submission for ANDES,2024-04-22 15:24:10,67.0
28,[0/seeking-editor],CyNetDiff Submission,2024-04-22 03:28:38,67.0
29,[1/editor-assigned],Submission for AMS,2024-04-22 01:37:56,67.0


In [ ]:
# TODO: add editor for each review? 
# TODO: list all current editors 

In [ ]:
# Get all currently closed / approved issues 

# calculate time that they were in review. 